# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [2]:
from azureml.core import Workspace,Experiment,Dataset
from azureml.core.compute import ComputeTarget,AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails
from azureml.train.automl import AutoMLConfig
from azureml.train.automl.utilities import get_primary_metrics

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [3]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'exp-heart1'

experiment=Experiment(ws, experiment_name)

In [5]:
try:
    cluster=ComputeTarget(workspace=ws,name="compute-1")
    print('cluster exists')
except:
    config =AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    cluster =ComputeTarget.create(ws,"compute-1",config)
    
cluster.wait_for_completion()
    

Provisioning operation finished, operation "Succeeded"


In [6]:
heart_data = Dataset.get_by_name(ws,name="heartfailure")
heart_data.take(5).to_pandas_dataframe()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [14]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 20,
    "task": "classification", 
    "primary_metric": "AUC_weighted",
    "training_data": heart_data,
    "label_column_name": "DEATH_EVENT",
    "n_cross_validations": 3,
    "enable_early_stopping": True,
    "debug_log" :"automl_errors.log",
    "max_concurrent_iterations": 4,
    "compute_target": cluster,
    "iterations":100
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(**automl_settings)

In [10]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
exp-heart1,AutoML_7864ded7-621a-42b1-a8e8-593d96f61793,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [11]:
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [12]:
remote_run.wait_for_completion(show_output=True)

Experiment,Id,Type,Status,Details Page,Docs Page
exp-heart1,AutoML_7864ded7-621a-42b1-a8e8-593d96f61793,automl,Running,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS

         1   MaxAbsScaler XGBoostClassifier                 0:00:48       0.8759    0.8759
         3   RobustScaler ExtremeRandomTrees                0:00:45       0.8705    0.8759
         0   MaxAbsScaler LightGBM                          0:00:50       0.8550    0.8759
         6   StandardScalerWrapper RandomForest             0:00:54       0.8919    0.8919
         7   MinMaxScaler ExtremeRandomTrees                0:00:51       0.8607    0.8919
         5   MinMaxScaler RandomForest                      0:02:53       0.8891    0.8919
         8   MinMaxScaler ExtremeRandomTrees                0:00:43       0.8552    0.8919
         9   SparseNormalizer KNN                           0:00:48       0.7429    0.8919
        10   MinMaxScaler ExtremeRandomTrees                0:00:59       0.8508    0.8919
        11   RobustScaler ExtremeRandomTrees                0:00:46       0.8656    0.8919
         2   MinMaxScaler RandomForest                      0:05:21       0.8932    0.8932

{'runId': 'AutoML_7864ded7-621a-42b1-a8e8-593d96f61793',
 'target': 'compute-1',
 'status': 'Completed',
 'startTimeUtc': '2021-04-22T04:57:39.674336Z',
 'endTimeUtc': '2021-04-22T05:24:21.845596Z',
 'properties': {'num_iterations': '100',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'AUC_weighted',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '3',
  'target': 'compute-1',
  'AMLSettingsJsonString': '{"path":null,"name":"exp-heart1","subscription_id":"a24a24d5-8d87-4c8a-99b6-91ed2d2df51f","resource_group":"aml-quickstarts-143334","workspace_name":"quick-starts-ws-143334","region":"southcentralus","compute_target":"compute-1","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"enable_run_restructure":false,"iterations":100,"primary_metric":"AUC_weighted","task_type":"classification","data_script":null,"test_size":0.0,"validation_size":0.0,"n

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [15]:
best_auto_run,best_auto_model=remote_run.get_output()
best_auto_model._final_estimator

PreFittedSoftVotingClassifier(classification_labels=None,
                              estimators=[('32',
                                           Pipeline(memory=None,
                                                    steps=[('standardscalerwrapper',
                                                            <azureml.automl.runtime.shared.model_wrappers.StandardScalerWrapper object at 0x7f6a53267f98>),
                                                           ('xgboostclassifier',
                                                            XGBoostClassifier(base_score=0.5,
                                                                              booster='gbtree',
                                                                              colsample_bylevel=1,
                                                                              colsample_bynode=1,
                                                                              colsample_bytree=0.9...
                 

In [16]:
#TODO: Save the best model
import joblib
joblib.dump(best_auto_model,"automl_model.pkl")

['automl_model.pkl']

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
from 

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service